# Basic/B1 Example
Import the `Geant4` and `Geant4.SystemOfUnits` modules

In [ ]:
using Geant4
using Geant4.SystemOfUnits

### Define the function that will construct the detector geometry.
The setup consists of a an envelope of box shape containing two volumes: a spherical cone and a trapezoid.

In this example we use  some common materials materials for medical applications. The envelope is made of water and the two inner volumes
are made from tissue and bone materials.
The materials are created with the help of the G4NistManager class,
which allows to build a material from the NIST database using their
names. All available materials can be found in the Geant4 User's Guide
for Application Developers, Appendix 10: Geant4 Materials Database.

In [ ]:
include("DetectorB1.jl")

detdesc = G4JLDetectorConstruction(constructB1Detector)
physics = QBBC()
#SetVerboseLevel(physics, 1)

### User Actions

In [ ]:
# User Actions
function buildApp(self::G4JLActionInitialization, ::Nothing)
    # Create particle gun
    particle_gun = G4JLParticleGun()  
    # Setup particle gun
    pg = GetGun(particle_gun)
    SetParticleByName(pg, "e-")
    SetParticleEnergy(pg, 100MeV)
    SetParticleMomentumDirection(pg, G4ThreeVector(0,0,1))
    SetParticlePosition(pg, G4ThreeVector(0,0,-16cm))
    # Register and relinquish ownership
    SetUserAction(self, move!(particle_gun))
  end
  app = G4JLActionInitialization(buildApp)

### Instantiate RunManager and initialize

In [ ]:
# Construct the default run manager and set initializers
runManager = G4RunManager()

# Register User initialization objects and relinquish ownership
SetUserInitialization(runManager, move!(detdesc))
SetUserInitialization(runManager, move!(physics))
SetUserInitialization(runManager, move!(app))

# Get the pointer to the User Interface manager
UImanager = G4UImanager!GetUIpointer();

### Execute the commands to initialize and run few events

In [ ]:
# Initialize kernel
ApplyCommand(UImanager, "/run/initialize")

# Change verbosity
#ApplyCommand(UImanager, "/control/verbose 2")
#ApplyCommand(UImanager, "/run/verbose 2")
#ApplyCommand(UImanager, "/event/verbose 0")
ApplyCommand(UImanager, "/tracking/verbose 1")

# gamma 6 MeV to the direction (0.,0.,1.)
ApplyCommand(UImanager, "/gun/particle gamma")
ApplyCommand(UImanager, "/gun/energy 6 MeV")

# go
ApplyCommand(UImanager, "/run/beamOn 1")

In [ ]:
ApplyCommand(UImanager, "/gun/particle e+")
ApplyCommand(UImanager, "/gun/energy 60 MeV")
ApplyCommand(UImanager, "/run/beamOn 1")